# Week 1. Project evaluation

## Case 1. Change of bank scoring for loans

To make decision about clients requests for loan bank uses scoring model that predicts the probability of default and potential economic value added.

Scoring allows to distinguish “bad” clients from “good” clients. The aim of a new AI model is to improve current model to swap customers that will default for “good” clients (“swap in – swap out” effect).

Bank is selling only one product – loans to individuals. Thus there is no cannibalization risk.



We need 6 month to build the model. 
Valuation period for the project is 3 years since the moment the model is created

In [1]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt


### Several constants

In [2]:
Income_Tax_Rate = 0.2
Cost_of_Equity = 0.1
Cost_of_Debt = 0.05
Equity_Weight = 0.15
Debt_Weight = 0.85

### WACC - Weight average cost of capital
$$
WACC= Cost of Equity × Equity Weight + Cost of Debt × Debt Weight × (1 - Income Tax Rate)
$$

In [3]:
WACC = Cost_of_Equity*Equity_Weight + Cost_of_Debt*Debt_Weight*(1 - Income_Tax_Rate)
WACC

0.049

### Inflation

Inflation factor for 1 - 12 months = 1

Inflation factor for 13 - 24 months	= 1.04

Inflation factor for 25 - 42 months = 1.08



In [4]:
df = pd.DataFrame()

In [5]:
df['period'] = range(0, 42)

- `ds_train` -  Number of employees to build a model 
- `ds_maintain` -  Number of employees to maintain a model 
- `ds_tratin_w` -  Workload of employees to build a model 
- `ds_maintain_w` -  Workload of employees to maintain a model 
- `ds_train_l` -  Labor costs on 1 employee to build a model 
- `ds_maintain_l` -  Labor costs on 1 employee to maintain a model 

In [6]:
# collect data and train model for 6 month
df['ds_train'] = 0
df['ds_train'].loc[range(0, 6)] = 12

# next 3 years 2 DS need to maintain a model (calibration, analize)
df['ds_maintain'] = 0
df['ds_maintain'].loc[range(6, 42)] = 2

ds_tratin_w = 1 # full-time
ds_maintain_w = 1 # full-time

ds_tratin_l = 1000
ds_maintain_l = 1000

In [7]:
# Labor costs to build a model
df['c_labour_train'] = ds_tratin_w * ds_tratin_l * df['ds_train']

# Labor costs to build a model
df['c_labour_maintain'] = ds_maintain_w * ds_maintain_l * df['ds_maintain']


In [8]:
# Other costs:
# Consulting (OPEX)
df['Consulting'] = 0
df['Consulting'].loc[range(0, 2)] = 10000

# Costs on IT resources (OPEX)
df['IT'] = 500

#Data purchase (OPEX)
df['Data'] = 1000


### Total costs

In [9]:
df['costs'] = (-1) * (1 - Income_Tax_Rate) * \
    (df['c_labour_train'] + df['c_labour_maintain'] + df['Consulting'] + df['IT'] + df['Data'])

In [10]:
df['inflation'] = 0
df['inflation'].loc[range(0,12)] = 1
df['inflation'].loc[range(12,24)] = 1.04
df['inflation'].loc[range(24,42)] = 1.08

In [11]:
df['costs_inf'] = df['costs'] * df['inflation']

In [12]:
print('Total costs ' ,  sum(df['costs_inf'] ))


Total costs  -186976.0


### Effect of sales growth

<table>
  <thead>
    <tr>
      <th> metric</th>
      <th>AS IS</th>
      <th>TO BE</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>Average sum (USD)</td>
      <td>1000</td>
      <td>1003</td>
    </tr>
    <tr>
      <td>Average rate %</td>
      <td>3%</td>
      <td>3%</td>
    </tr>
    <tr>
      <td>Average term (months)</td>
      <td>23.8</td>
      <td>24</td>
    </tr>
    <tr>
      <td>Average expected losses (%)</td>
      <td>1%</td>
      <td>0.9%</td>
    </tr>
    <tr>
      <td>Average cost of funding (%)</td>
      <td>0.5%</td>
      <td>0.5%</td>
    </tr>
    <tr>
      <td>Average NPV of 1 sale (USD)</td>
      <td>4</td>
      <td>4.4</td>
    </tr>
    <tr>
      <td>Approval rate (%)</td>
      <td>20</td>
      <td>20%</td>
    </tr>
    <tr>
      <td>Take up rate (%)</td>
      <td>50%</td>
      <td>50%</td>
    </tr>
  </tbody>
</table>

The effect of sales growth is calculated using the following formula
$$ 
Effect = NPV_{TO - BE} × Sales - NPV_{AS - IS} × Sales
$$

In [13]:
NPV_as_is = 4
NPV_to_be = 4.4

### Sales plan AS-IS = Sales plan TO-BE
Sales  for 7 - 12 months = 10000

Sales  for 13 - 24 months = 20000

Sales  for 25 - 42 months = 30000

In [14]:
df['sales'] = 0
df['sales'].loc[range(6,12)] = 10000
df['sales'].loc[range(12,24)] = 20000
df['sales'].loc[range(24,42)] = 30000

In [15]:
df['Benefit_inf'] = df['sales']* (NPV_to_be - NPV_as_is) * df['inflation']

In [16]:
print('Benefit  ' ,  sum(df['Benefit_inf']) )

Benefit   357120.00000000006


### Discounted cash flows 

#### Now all that we need is to build discounted cash flows as a difference between total costs and effect of sales growth.

$$
Discount Coeficient_i = \frac{1}{( 1 + WACC) ^ \frac{i}{12} }
$$

In [17]:
df['discount_coef'] = 1 / (1 + WACC) ** ((df['period'] + 1)/12)

In [18]:
df['cash_flow'] = df['Benefit_inf'] +  df['costs_inf']
df['cash_flow_discount'] =  df['discount_coef'] * df['cash_flow'] 

In [19]:
NPV = sum(df['cash_flow_discount'] )

In [20]:
print('NPV   ' ,  round(NPV, 1) )

NPV    143935.4


### So we can say that this AI initiative is profitable (NPV>0) and we can accept this project